### Scraper Oscar: Raspando tables com Pandas

Com o **Pandas** é possível raspar tabelas simples de sites em HTML. Neste notebook vamos raspar tabelas do **Wikipédia** de algumas das categorias da premiação e análisar seus dados. Repositório original da Judite Cypreste em <https://github.com/juditecypreste/scraper_oscar> 

In [1]:
%pip install pandas requests numpy lxml beautifulsoup4 html5lib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import numpy as np
import io

#### Categoria: Melhor Atriz

Para conseguirmos as vencedoras e indicadas ao prêmio de *Melhor Atriz* o processo é simples, já que a estrutura da página [Wikipédia](https://pt.wikipedia.org/wiki/Oscar_de_melhor_atriz) sobre o assunto está bem organizada. 

Vamos então usar 
`pd.read_html` para ler a url.

In [3]:
url_wiki = "https://pt.wikipedia.org/wiki/Oscar_de_melhor_atriz"
headers = {"User-Agent": "Mozilla/5.0"}

resp_wiki = requests.get(url_wiki, headers=headers)
resp_wiki.raise_for_status()

# Essa página tem mais de uma tabela; a que queremos é a com colunas Film / Year / Awards / Nominations
tables_wiki = pd.read_html(io.StringIO(resp_wiki.text))

filtered_tables = [df for df in tables_wiki if 'Vencedores e Indicados' in df.columns]
filtered_tables

[          Ano Vencedores e Indicados             Personagem  \
 0  1929 (1.ª)           Janet Gaynor  Diane Angela A Esposa   
 1  1929 (1.ª)         Louise Dresser           Mrs. Pleznik   
 2  1929 (1.ª)         Gloria Swanson         Sadie Thompson   
 
                                                Filme Ref.  
 0  7th Heaven Street Angel Sunrise: A Song of Two...  [7]  
 1                                    A Ship Comes In  [7]  
 2                                     Sadie Thompson  [7]  ,
                     Ano Vencedores e Indicados  \
 0   1930 (2.ª) [nota 1]          Mary Pickford   
 1   1930 (2.ª) [nota 1]        Ruth Chatterton   
 2   1930 (2.ª) [nota 1]          Betty Compson   
 3   1930 (2.ª) [nota 1]          Jeanne Eagels   
 4   1930 (2.ª) [nota 1]       Corinne Griffith   
 5   1930 (2.ª) [nota 1]            Bessie Love   
 6            1931 (3.ª)  Norma Shearer[nota 2]   
 7            1931 (3.ª)          Nancy Carroll   
 8            1931 (3.ª)        Ruth C

Antes de procurar a tabela que queremos é preciso saber que a numeração em Python começa a partir do zero. Tendo isso em mente, precisamos procurar a tabela certa com as informações que queremos. No nosso caso a tabela que queremos é a segunda que aparece na página, logo é a de *número 1*.  

Lembre-se a contagem **sempre** começa a partir do número zero.

In [4]:
tabela_atriz = pd.concat(filtered_tables, ignore_index=True)
tabela_atriz

,Ano,Vencedores e Indicados,Personagem,Filme,Ref.
0,1929 (1.ª),Janet Gaynor,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...,[7]
1,1929 (1.ª),Louise Dresser,Mrs. Pleznik,A Ship Comes In,[7]
2,1929 (1.ª),Gloria Swanson,Sadie Thompson,Sadie Thompson,[7]
3,1930 (2.ª) [nota 1],Mary Pickford,Norma Besant,Coquette,[8]
4,1930 (2.ª) [nota 1],Ruth Chatterton,Jacqueline Floriot,Madame X,[8]
...,...,...,...,...,...
481,2026 (98.ª),Jessie Buckley,Agnes Shakespeare,Hamnet,NaN
482,2026 (98.ª),Rose Byrne,Linda,"If I Had Legs, I'd Kick You",NaN
483,2026 (98.ª),Kate Hudson,Claire Sardina,Song Sung Blue,NaN
484,2026 (98.ª),Renate Reinsve,Nora Borg,Affeksjonsverdi,NaN


Note que na raspagem temos as seguintes colunas: *'Ano', 'Atriz', 'Papel', 'Filme' e 'Ref'*.  
Não vamos usar a coluna *'Ref'* então vamos excluir de nosso dataframe.

In [5]:
# excluindo coluna 'Ref'
tabela_atriz.drop('Ref.', axis=1, inplace=True)
tabela_atriz

,Ano,Vencedores e Indicados,Personagem,Filme
0,1929 (1.ª),Janet Gaynor,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...
1,1929 (1.ª),Louise Dresser,Mrs. Pleznik,A Ship Comes In
2,1929 (1.ª),Gloria Swanson,Sadie Thompson,Sadie Thompson
3,1930 (2.ª) [nota 1],Mary Pickford,Norma Besant,Coquette
4,1930 (2.ª) [nota 1],Ruth Chatterton,Jacqueline Floriot,Madame X
...,...,...,...,...
481,2026 (98.ª),Jessie Buckley,Agnes Shakespeare,Hamnet
482,2026 (98.ª),Rose Byrne,Linda,"If I Had Legs, I'd Kick You"
483,2026 (98.ª),Kate Hudson,Claire Sardina,Song Sung Blue
484,2026 (98.ª),Renate Reinsve,Nora Borg,Affeksjonsverdi


Se olharmos a primeira coluna, a de *Ano*, veremos que ela tem uma sujeira. Precisamos tirar ela para padronizar nossos dados.   
Como estamos lidando com strings de tamanhos diferentes, vamos usar uma separação de strings.  

Com o `str[0]` nos selecionamos qual das strings queremos. Lembre-se que a contagem começa no zero.

In [6]:
# tirando a sujeira da coluna 'Ano'
tabela_atriz['Ano'] = tabela_atriz.Ano.str.split().str[0]
tabela_atriz

,Ano,Vencedores e Indicados,Personagem,Filme
0,1929,Janet Gaynor,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...
1,1929,Louise Dresser,Mrs. Pleznik,A Ship Comes In
2,1929,Gloria Swanson,Sadie Thompson,Sadie Thompson
3,1930,Mary Pickford,Norma Besant,Coquette
4,1930,Ruth Chatterton,Jacqueline Floriot,Madame X
...,...,...,...,...
481,2026,Jessie Buckley,Agnes Shakespeare,Hamnet
482,2026,Rose Byrne,Linda,"If I Had Legs, I'd Kick You"
483,2026,Kate Hudson,Claire Sardina,Song Sung Blue
484,2026,Renate Reinsve,Nora Borg,Affeksjonsverdi


In [7]:
tabela_atriz.rename(columns={'Vencedores e Indicados':'Atriz'}, inplace=True)

Na coluna com os nomes também existe uma sujeira: as notas explicativas que acompanham algumas atrizes. Vamos eliminar isso também, mas de uma forma diferente já que, ao contrário do exemplo anterior, as palavras não estão separadas por um espaço. 

Procurando um padrão, vemos que as notas estão entre **colchetes [  ]**. É a partir de padrões que podemos estabalecer como podemos limpar nossos dados.  

Vamos então usar isso ao nosso favor.

Vamos começar separando os nomes em uma coluna e as notas em outra coluna.   
Criamos a coluna **Nome** e a coluna **Nota**. Perceba que onde não tinha nenhuma nota o Pandas incluiu um **None**.  

Olhe também a nova coluna **Nome** criada e veja que ela já não tem notas.

In [8]:
# dividindo os nomes das notas.
tabela_atriz[['Nome','Nota']] = tabela_atriz.Atriz.str.split("[",expand=True)
tabela_atriz

,Ano,Atriz,Personagem,Filme,Nome,Nota
0,1929,Janet Gaynor,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...,Janet Gaynor,NaN
1,1929,Louise Dresser,Mrs. Pleznik,A Ship Comes In,Louise Dresser,NaN
2,1929,Gloria Swanson,Sadie Thompson,Sadie Thompson,Gloria Swanson,NaN
3,1930,Mary Pickford,Norma Besant,Coquette,Mary Pickford,NaN
4,1930,Ruth Chatterton,Jacqueline Floriot,Madame X,Ruth Chatterton,NaN
...,...,...,...,...,...,...
481,2026,Jessie Buckley,Agnes Shakespeare,Hamnet,Jessie Buckley,NaN
482,2026,Rose Byrne,Linda,"If I Had Legs, I'd Kick You",Rose Byrne,NaN
483,2026,Kate Hudson,Claire Sardina,Song Sung Blue,Kate Hudson,NaN
484,2026,Renate Reinsve,Nora Borg,Affeksjonsverdi,Renate Reinsve,NaN


In [9]:
# Excluindo as colunas 'Atrizes' e 'Nota'.
tabela_atriz.drop(['Atriz','Nota'], axis=1, inplace=True)
tabela_atriz

,Ano,Personagem,Filme,Nome
0,1929,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...,Janet Gaynor
1,1929,Mrs. Pleznik,A Ship Comes In,Louise Dresser
2,1929,Sadie Thompson,Sadie Thompson,Gloria Swanson
3,1930,Norma Besant,Coquette,Mary Pickford
4,1930,Jacqueline Floriot,Madame X,Ruth Chatterton
...,...,...,...,...
481,2026,Agnes Shakespeare,Hamnet,Jessie Buckley
482,2026,Linda,"If I Had Legs, I'd Kick You",Rose Byrne
483,2026,Claire Sardina,Song Sung Blue,Kate Hudson
484,2026,Nora Borg,Affeksjonsverdi,Renate Reinsve


Para deixar nossos dados padronizados vamos mudar o nome da coluna **Nome** para **Atriz**.

In [10]:
# Mudando none da coluna 'Nome' para 'Atriz'.
tabela_atriz.rename(columns={'Nome':'Atriz'}, inplace=True)
tabela_atriz

,Ano,Personagem,Filme,Atriz
0,1929,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...,Janet Gaynor
1,1929,Mrs. Pleznik,A Ship Comes In,Louise Dresser
2,1929,Sadie Thompson,Sadie Thompson,Gloria Swanson
3,1930,Norma Besant,Coquette,Mary Pickford
4,1930,Jacqueline Floriot,Madame X,Ruth Chatterton
...,...,...,...,...
481,2026,Agnes Shakespeare,Hamnet,Jessie Buckley
482,2026,Linda,"If I Had Legs, I'd Kick You",Rose Byrne
483,2026,Claire Sardina,Song Sung Blue,Kate Hudson
484,2026,Nora Borg,Affeksjonsverdi,Renate Reinsve


E também organizar a ordem de nossas colunas.

In [11]:
# organizando as colunas 
melhor_atriz = tabela_atriz[['Ano','Atriz','Personagem','Filme']]
melhor_atriz

,Ano,Atriz,Personagem,Filme
0,1929,Janet Gaynor,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...
1,1929,Louise Dresser,Mrs. Pleznik,A Ship Comes In
2,1929,Gloria Swanson,Sadie Thompson,Sadie Thompson
3,1930,Mary Pickford,Norma Besant,Coquette
4,1930,Ruth Chatterton,Jacqueline Floriot,Madame X
...,...,...,...,...
481,2026,Jessie Buckley,Agnes Shakespeare,Hamnet
482,2026,Rose Byrne,Linda,"If I Had Legs, I'd Kick You"
483,2026,Kate Hudson,Claire Sardina,Song Sung Blue
484,2026,Renate Reinsve,Nora Borg,Affeksjonsverdi


Por fim, vamos salvar nossos dados em um csv. 

In [12]:
# salvando em csv
melhor_atriz.to_csv('melhor_atriz.csv')
melhor_atriz

,Ano,Atriz,Personagem,Filme
0,1929,Janet Gaynor,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...
1,1929,Louise Dresser,Mrs. Pleznik,A Ship Comes In
2,1929,Gloria Swanson,Sadie Thompson,Sadie Thompson
3,1930,Mary Pickford,Norma Besant,Coquette
4,1930,Ruth Chatterton,Jacqueline Floriot,Madame X
...,...,...,...,...
481,2026,Jessie Buckley,Agnes Shakespeare,Hamnet
482,2026,Rose Byrne,Linda,"If I Had Legs, I'd Kick You"
483,2026,Kate Hudson,Claire Sardina,Song Sung Blue
484,2026,Renate Reinsve,Nora Borg,Affeksjonsverdi


In [16]:
# Código para transformar anos em décadas no DataFrame de atrizes do Oscar

# Exemplo assumindo que você já tem o DataFrame 'tabela_atrizes' carregado
# (conforme mostrado no notebook com pd.read_html)

# Função para converter ano em década
def ano_para_decada(ano_str):
    """
    Converte um ano em formato string para sua respectiva década.
    
    Exemplos:
    - '1929 (1.ª)' -> '1920s'
    - '1930 (2.ª)[nota 1]' -> '1930s'
    - '2020' -> '2020s'
    """
    # Extrai apenas os 4 primeiros dígitos do ano
    ano = int(str(ano_str)[:4])
    
    # Calcula a década (arredonda para baixo para o múltiplo de 10)
    decada = (ano // 10) * 10
    
    return decada

# Aplicando a transformação
# Método 1: Criar uma nova coluna 'Década'
tabela_atriz['Década'] = tabela_atriz['Ano'].apply(ano_para_decada)

# Método 2 (alternativo): Substituir a coluna 'Ano' pela década
# tabela_atrizes['Ano'] = tabela_atrizes['Ano'].apply(ano_para_decada)

# Visualizando o resultado
# print(tabela_atriz[['Ano', 'Década', 'Atriz', 'Filme']].head(20))
tabela_atriz

,Ano,Personagem,Filme,Atriz,Década
0,1929,Diane Angela A Esposa,7th Heaven Street Angel Sunrise: A Song of Two...,Janet Gaynor,1920
1,1929,Mrs. Pleznik,A Ship Comes In,Louise Dresser,1920
2,1929,Sadie Thompson,Sadie Thompson,Gloria Swanson,1920
3,1930,Norma Besant,Coquette,Mary Pickford,1930
4,1930,Jacqueline Floriot,Madame X,Ruth Chatterton,1930
...,...,...,...,...,...
481,2026,Agnes Shakespeare,Hamnet,Jessie Buckley,2020
482,2026,Linda,"If I Had Legs, I'd Kick You",Rose Byrne,2020
483,2026,Claire Sardina,Song Sung Blue,Kate Hudson,2020
484,2026,Nora Borg,Affeksjonsverdi,Renate Reinsve,2020


In [18]:
# Exemplo de análise: contagem de nomeações por década
por_decada = tabela_atriz['Década'].value_counts().sort_index()
por_decada

Década
1920     3
1930    48
1940    50
1950    50
1960    50
1970    50
1980    50
1990    50
2000    50
2010    50
2020    35
Name: count, dtype: int64